In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!ls ../"00. Data"

PP_recipes.csv              images.csv
PP_users.csv                ingr_map.pkl
RAW_interactions.csv        interactions_train.csv
RAW_recipes.csv             interactions_validation.csv


In [3]:
data = pd.read_csv('../00. Data/RAW_recipes.csv')
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
data.shape

(231637, 12)

In [5]:
pd.options.display.max_colwidth = 1000

In [6]:
# Initial image_df creation:
# image_df = data[['id']].sort_values(by="id").reset_index().drop("index", axis=1)
# image_df['image_url'] = ""

image_df = pd.read_csv('../00. Data/images.csv')
image_df.head()

,id,image_url
0,38,"https://img.sndimg.com:443/food/image/upload/c_thumb,q_80,w_721,h_406/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229 berry blue frzn dess.jpg"
1,39,"https://img.sndimg.com:443/food/image/upload/c_thumb,q_80,w_721,h_406/v1/img/recipes/39/picM9Mhnw.jpg"
2,40,"https://img.sndimg.com:443/food/image/upload/c_thumb,q_80,w_721,h_406/v1/img/recipes/40/picJ4Sz3N.jpg"
3,41,"https://img.sndimg.com:443/food/image/upload/c_thumb,q_80,w_721,h_406/v1/img/recipes/41/picmbLig8.jpg"
4,43,NaN


In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import chromedriver_binary

In [13]:
base_url = "https://www.food.com/recipe/"
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

start = image_df.loc[image_df['image_url'][::-1].notna().idxmax(), 'id']
i = image_df[image_df['id'] <= start].shape[0]

for index, rid in enumerate(image_df['id']):
    
    if rid <= start:
        continue
    
    try:
        driver.get(base_url + str(rid).strip()) 
        wait = WebDriverWait(driver, 15)
        wait.until(ec.visibility_of_element_located((By.XPATH, "//img[@class='recipe-image__img']")))
        
    except TimeoutException as ex:
        print(f"No recipe #{rid} detected, {round(i / image_df.shape[0] * 100, 2)}% completed")
        continue
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    if soup is not None:
        div = soup.find("div", class_="recipe-hero")
        
        if div is not None:
            img = div.find("img", class_='recipe-image__img')
    
            if img is not None and 'data-src' in img.attrs:
                image_df.loc[index, 'image_url'] = img['data-src']
                print(f"Imported recipe #{rid}, {round(i / image_df.shape[0] * 100, 2)}% completed")
                
            else:
                print(f"No photo for recipe #{rid}, {round(i / image_df.shape[0] * 100, 2)}% completed")
   
        else:
            print(f"No hero for recipe #{rid}, {round(i / image_df.shape[0] * 100, 2)}% completed")

    else:
        print(f"No soup for recipe #{rid}, {round(i / image_df.shape[0] * 100, 2)}% completed")
    
    i += 1
    

No hero for recipe #48916, 11.82% completed
No hero for recipe #48917, 11.82% completed
No hero for recipe #48919, 11.82% completed
No hero for recipe #48920, 11.82% completed
No hero for recipe #48921, 11.82% completed
No hero for recipe #48924, 11.82% completed
No hero for recipe #48925, 11.82% completed
Imported recipe #48930, 11.82% completed
No hero for recipe #48931, 11.82% completed
No hero for recipe #48933, 11.83% completed
No hero for recipe #48934, 11.83% completed
Imported recipe #48936, 11.83% completed
Imported recipe #48938, 11.83% completed
No hero for recipe #48939, 11.83% completed
No hero for recipe #48940, 11.83% completed
No hero for recipe #48942, 11.83% completed
Imported recipe #48943, 11.83% completed
No hero for recipe #48944, 11.83% completed
Imported recipe #48946, 11.83% completed
Imported recipe #48948, 11.83% completed
No hero for recipe #48950, 11.83% completed
No hero for recipe #48951, 11.83% completed
No hero for recipe #48952, 11.83% completed
No her

Imported recipe #49247, 11.9% completed
Imported recipe #49248, 11.91% completed
Imported recipe #49249, 11.91% completed
No hero for recipe #49250, 11.91% completed
Imported recipe #49252, 11.91% completed
Imported recipe #49254, 11.91% completed
No hero for recipe #49258, 11.91% completed
No hero for recipe #49262, 11.91% completed
No hero for recipe #49265, 11.91% completed
No hero for recipe #49267, 11.91% completed
No hero for recipe #49268, 11.91% completed
No hero for recipe #49270, 11.91% completed
Imported recipe #49271, 11.91% completed
No hero for recipe #49274, 11.91% completed
Imported recipe #49275, 11.91% completed
No hero for recipe #49276, 11.91% completed
No hero for recipe #49277, 11.91% completed
Imported recipe #49278, 11.91% completed
Imported recipe #49284, 11.91% completed
Imported recipe #49287, 11.91% completed
No hero for recipe #49289, 11.91% completed
No hero for recipe #49290, 11.91% completed
No hero for recipe #49294, 11.91% completed
No hero for recipe 

No hero for recipe #49590, 11.99% completed
Imported recipe #49591, 11.99% completed
No hero for recipe #49593, 11.99% completed
No hero for recipe #49594, 11.99% completed
Imported recipe #49595, 11.99% completed
Imported recipe #49596, 11.99% completed
No hero for recipe #49597, 11.99% completed
No hero for recipe #49599, 11.99% completed
No hero for recipe #49600, 11.99% completed
Imported recipe #49602, 11.99% completed
No hero for recipe #49605, 11.99% completed
No hero for recipe #49611, 11.99% completed
Imported recipe #49612, 11.99% completed
Imported recipe #49613, 11.99% completed
No hero for recipe #49615, 11.99% completed
No hero for recipe #49616, 11.99% completed
No hero for recipe #49618, 12.0% completed
Imported recipe #49619, 12.0% completed
No hero for recipe #49620, 12.0% completed
No hero for recipe #49624, 12.0% completed
No hero for recipe #49634, 12.0% completed
No hero for recipe #49636, 12.0% completed
No hero for recipe #49638, 12.0% completed
Imported recipe 

No hero for recipe #49941, 12.07% completed
Imported recipe #49942, 12.07% completed
Imported recipe #49943, 12.07% completed
No hero for recipe #49944, 12.07% completed
Imported recipe #49945, 12.07% completed
No hero for recipe #49947, 12.07% completed
Imported recipe #49950, 12.07% completed
No hero for recipe #49952, 12.07% completed
No hero for recipe #49953, 12.07% completed
No hero for recipe #49955, 12.07% completed
Imported recipe #49956, 12.07% completed
Imported recipe #49963, 12.08% completed
Imported recipe #49964, 12.08% completed
No hero for recipe #49965, 12.08% completed
No hero for recipe #49969, 12.08% completed
No hero for recipe #49970, 12.08% completed
No hero for recipe #49971, 12.08% completed
No hero for recipe #49973, 12.08% completed
No hero for recipe #49974, 12.08% completed
Imported recipe #49976, 12.08% completed
No hero for recipe #49977, 12.08% completed
No hero for recipe #49978, 12.08% completed
Imported recipe #49980, 12.08% completed
Imported recipe

No hero for recipe #50278, 12.15% completed
No hero for recipe #50279, 12.15% completed
No hero for recipe #50280, 12.15% completed
Imported recipe #50281, 12.15% completed
No hero for recipe #50282, 12.16% completed
No hero for recipe #50283, 12.16% completed
Imported recipe #50284, 12.16% completed
No hero for recipe #50285, 12.16% completed
No hero for recipe #50286, 12.16% completed
No hero for recipe #50287, 12.16% completed
No hero for recipe #50288, 12.16% completed
No hero for recipe #50289, 12.16% completed
No hero for recipe #50290, 12.16% completed
Imported recipe #50291, 12.16% completed
No hero for recipe #50292, 12.16% completed
No hero for recipe #50295, 12.16% completed
No hero for recipe #50296, 12.16% completed
No hero for recipe #50299, 12.16% completed
No hero for recipe #50301, 12.16% completed
Imported recipe #50302, 12.16% completed
Imported recipe #50303, 12.16% completed
No hero for recipe #50304, 12.16% completed
No hero for recipe #50305, 12.16% completed
No 

KeyboardInterrupt: 

In [14]:
image_df.to_csv('../00. Data/images.csv', index = False)

In [ ]:
image_df.head(10)

In [ ]:
f"{round(len(image_df[image_df['image_url'] != '']) / len(image_df), 4) * 100}% with data"

In [ ]:
url = image_df[image_df['image_url'] != ""].sample()['image_url']
url

In [ ]:
# import webbrowser
# webbrowser.open(url)

# ![Test](https://img.sndimg.com:443/food/image/upload/c_thumb,q_80,w_721,h_406/v1/img/recipes/40/picJ4Sz3N.jpg)